In [15]:
from langchain.chains import APIChain
from libs import *
from dotenv import load_dotenv
from prompt_improved import *
import httpx
import logging as logger
import sys

# Set WARNING Logger levels help print only meaningful text
logger.basicConfig(stream=sys.stdout, level=logger.WARNING)
logger.getLogger('botocore').setLevel(logger.WARNING)
logger.getLogger('httpx').setLevel(logger.WARNING)

# loading ENV variables
load_dotenv()

# Initialize Models
gpt35 = create_llm(model={"provider":"OpenAI", "value": "gpt-3.5-turbo"}, model_kwargs={"temperature": 0.1})
gpt4 = create_llm(model={"provider":"OpenAI", "value": "gpt-4"}, model_kwargs={"temperature": 0.1})
claude3 = create_llm(model={"provider":"Anthropic", "value": "anthropic.claude-3-sonnet-20240229-v1:0"}, model_kwargs={"temperature": 0.1})
llama2 = create_llm(model={"provider":"Meta", "value": "meta.llama2-70b-chat-v1"}, model_kwargs=None)

# Dynamically retrieve swagger
output = httpx.get("https://www.klarna.com/us/shopping/public/openai/v0/api-docs/")
swagger = output.text

# build the APIChain 
chain = APIChain.from_llm_and_api_docs(
    llm=gpt4,
    api_docs=swagger,
    verbose=False,
    limit_to_domains=["klarna.com", "https://www.klarna.com/", "https://www.klarna.com"],
    api_url_prompt=FINE_TUNED_API_URL_PROMPT,
    api_response_prompt=FINE_TUNED_API_RESPONSE_PROMPT
)


# Ask a question to the Chain
response = chain.invoke(
    "Find 3 t-shirts, max 50 USD. For each Product print the Description, the Price and the corresponding URL"
)

# Print the Chain Output
print(response['output'])

1. Product: Polo Ralph Lauren Men's Slim Fit Wicking Crew Undershirts 3-pack - White
   Price: $37.99
   URL: [Product Link](https://www.klarna.com/us/shopping/pl/cl10001/3207134809/Clothing/Polo-Ralph-Lauren-Men-s-Slim-Fit-Wicking-Crew-Undershirts-3-pack-White/?utm_source=openai&ref-site=openai_plugin)

2. Product: Lacoste Men's T-shirts 3-pack - Black
   Price: $31.90
   URL: [Product Link](https://www.klarna.com/us/shopping/pl/cl10001/3202043025/Clothing/Lacoste-Men-s-T-shirts-3-pack-Black/?utm_source=openai&ref-site=openai_plugin)

3. Product: SKIMS Cotton Jersey T-shirt
   Price: $48.00
   URL: [Product Link](https://www.klarna.com/us/shopping/pl/cl10001/3202929904/Clothing/SKIMS-Cotton-Jersey-T-shirt/?utm_source=openai&ref-site=openai_plugin)


In [3]:
output = httpx.get("https://raw.githubusercontent.com/open-meteo/open-meteo/main/openapi.yml")
meteo_swagger = output.text

chain = APIChain.from_llm_and_api_docs(
    llm=llama2,
    api_docs=meteo_swagger,
    verbose=True,
    limit_to_domains=None,
    api_url_prompt=FINE_TUNED_API_URL_PROMPT,
    api_response_prompt=FINE_TUNED_API_RESPONSE_PROMPT
)

# Ask a question to the Chain
response = chain.invoke(
    "What is the weather like right now in Munich, Germany in degrees Fahrenheit?"
)

# Print the Chain Output
print(response['output'])



> Entering new APIChain chain...
  https://api.open-meteo.com/v1/forecast?hourly=true&temperature_unit=fahrenheit&wind_speed_unit=mph&timeformat=unixtime&latitude=48.137399&longitude=11.575193&current_weather=true
{"error":true,"reason":"Data corrupted at path ''. Cannot initialize SurfaceAndPressureVariable<VariableAndPreviousDay, ForecastPressureVariable> from invalid String value true."}

> Finished chain.
  The API call returned an error message indicating that the data is corrupted and cannot be initialized. Therefore, the question about the current weather in Munich, Germany cannot be answered using this API call.


In [5]:

chain = APIChain.from_llm_and_api_docs(
    llm=gpt35,
    api_docs=meteo_swagger,
    verbose=True,
    limit_to_domains=None,
    api_url_prompt=FINE_TUNED_API_URL_PROMPT,
    api_response_prompt=FINE_TUNED_API_RESPONSE_PROMPT
)

# Ask a question to the Chain
response = chain.invoke(
    "What is the weather like right now in Munich, Germany in degrees Fahrenheit?"
)

# Print the Chain Output
print(response['output'])



> Entering new APIChain chain...
https://api.open-meteo.com/v1/forecast?latitude=48.137&longitude=11.575&current_weather=true&temperature_unit=fahrenheit
{"latitude":48.14,"longitude":11.58,"generationtime_ms":0.07104873657226562,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":524.0,"current_weather_units":{"time":"iso8601","interval":"seconds","temperature":"°F","windspeed":"km/h","winddirection":"°","is_day":"","weathercode":"wmo code"},"current_weather":{"time":"2024-04-24T12:15","interval":900,"temperature":45.7,"windspeed":17.7,"winddirection":264,"is_day":1,"weathercode":3}}

> Finished chain.
The current weather in Munich, Germany is 45.7°F with a wind speed of 17.7 km/h coming from 264° direction.
